# Demo with a vlm dataset (openbmb/RLAIF-V-Dataset)

In [ ]:
!pip install -q kagglehub

!pip install -q tensorflow
!pip install -q tensorboardX
!pip install -q grain
# !pip install -q git+https://github.com/google/tunix
!pip install -q git+https://github.com/google/qwix

!pip uninstall -q -y flax
!pip install -q git+https://github.com/google/flax.git

!pip install -q huggingface_hub
!pip install -q datasets

In [ ]:
%cd /content
!git clone --branch uiuc-vlm --single-branch https://github.com/PLAN-Lab/tunix.git


In [ ]:
%cd /content/tunix
!pip -q install -e .

In [ ]:
# --- autoreload for dev loop ---
import sys, types, importlib
if 'imp' not in sys.modules:
    imp = types.ModuleType('imp')
    imp.reload = importlib.reload
    sys.modules['imp'] = imp

%load_ext autoreload
%autoreload 2

# sanity check import
import tunix
print("tunix imported from:", tunix.__file__)

In [ ]:
import functools
import gc
import os
from pprint import pprint
import re
import time

from flax import nnx
import grain
import humanize
import jax
import jax.numpy as jnp
import kagglehub
import optax
from orbax import checkpoint as ocp
import qwix
import tensorflow_datasets as tfds
from tqdm.auto import tqdm

from tunix.examples.data import translation_dataset as data_lib
from tunix.generate import sampler as sampler_lib
from tunix.generate.vlm_sampler import VLMSampler
from tunix.models.gemma3 import params as params_lib
from tunix.models.gemma3 import params_safetensors as params_safetensors_lib

from tunix.sft import metrics_logger
from datasets import load_dataset
from tunix.sft.dpo.dpo_trainer import DpoTrainingConfig
from tunix.sft.dpo.dpo_trainer import DpoTrainer
from tunix.sft.dpo.dpo_trainer import TrainingInput
from huggingface_hub import snapshot_download
from tunix.sft.dpo.dpo_trainer import _generate_ids_and_masks
from tunix.models.gemma3 import model as gemma3_model_lib
from datasets import concatenate_datasets

In [ ]:
# ====== Data ======
TRAIN_DATA_DIR = "./data/train"
TEST_DATA_DIR = "./data/test"
TRAIN_FRACTION = 1.0

INTERMEDIATE_CKPT_DIR = "/content/intermediate_ckpt/"
# ====== LoRA ======
RANK = 8
ALPHA = 16.0

# ====== Sharding ======
MESH = [(1, 1), ("fsdp", "tp")]

MAX_PROMPT_LENGTH = 192
TOTAL_GENERATION_STEPS = 192
TEMPERATURE = 0.7
TOP_P = 1.0
TOP_K = 50
BETA = 0.1

# === AdamW, warmup, cosine scheduler ===
LEARNING_RATE = 3e-6
B1 = 0.9
B2 = 0.99
WEIGHT_DECAY = 0.1

# == Cosine decay with warmup scheduler ==
# Linearly increase learning rate from 0. to 5e-6 in the first 10% training
# steps, and then gradually decrease the learning rate to 0 using cosine
# scheduler.
BATCH_SIZE = 4
NUM_BATCHES = 512
NUM_TEST_BATCHES = 100
EVAL_EVERY_N_STEPS = 100

NUM_EPOCHS = 1  # can potentially train for more epochs
TRAIN_FRACTION = 1.0
MAX_STEPS = int(NUM_BATCHES * TRAIN_FRACTION * NUM_EPOCHS)

WARMUP_STEPS = 0.1 * MAX_STEPS
# == Grad clipping ==
# Grad clipping to prevent large gradients. Found this
# important to keep KL divergence in check.
MAX_GRAD_NORM = 0.1

# ====== Inference ======
GENERATION_CONFIGS = {
    # greedy search
    "greedy": {"temperature": 1e-4, "top_k": 1, "top_p": 1.0},
    # some randomness
    "standard": {"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    # liberal
    "liberal": {"temperature": 0.85, "top_k": 2000, "top_p": 1.0},
}

In [ ]:

from huggingface_hub import login, HfApi
import os

HF_TOKEN = "xxx"  # <-- paste your Read token



In [ ]:
model_id = "google/gemma-3-1b-it"
ignore_patterns = [
    "*.pth",  # Ignore PyTorch .pth weight files
]
print(f"Downloading {model_id} from Hugging Face...")
local_model_path = snapshot_download(
    repo_id=model_id, ignore_patterns=ignore_patterns, token=HF_TOKEN
)
print(f"Model successfully downloaded to: {local_model_path}")

In [ ]:
MODEL_CP_PATH = local_model_path

model_config = (
    gemma3_model_lib.Gemma3Config.gemma3_1b()
)  # pick correponding config based on model version
MESH = [(1, 1), ("fsdp", "tp")]
mesh = jax.make_mesh(*MESH)
with mesh:
  gemma3 = params_safetensors_lib.create_model_from_safe_tensors(
      MODEL_CP_PATH, model_config, mesh
  )
  nnx.display(gemma3)

In [ ]:
# ==== 3) Load only the tokenizer first (cheap), NOT the whole processor yet ====
from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained("google/paligemma-3b-mix-224", token=HF_TOKEN)
print("Tokenizer loaded. pad_id:", tok.pad_token_id, "eos_id:", tok.eos_token_id)

# Adapter that your sampler expects
class HFTokenizerAdapter:
    def __init__(self, hf_tok):
        self.tok = hf_tok
        self._pad_id = self.tok.pad_token_id if self.tok.pad_token_id is not None else self.tok.eos_token_id
        self._eos_id = self.tok.eos_token_id
    def encode(self, s: str):
        return self.tok(s, add_special_tokens=False)["input_ids"]
    def decode(self, ids):
        return self.tok.decode(ids, skip_special_tokens=True)
    def pad_id(self) -> int: return int(self._pad_id)
    def eos_id(self) -> int: return int(self._eos_id)

gemma_tokenizer = HFTokenizerAdapter(tok)
print("Tokenizer adapter ready ✔️")

image_size = 224
print("Processor loaded. image_size:", image_size)

# ==== 5) Construct the sampler LAST (avoid touching model state in __init__) ====
# IMPORTANT: gemma3 must be your PaLI-Gemma nnx Module (not a text-only Gemma).
# If it's named differently, change transformer=... accordingly.
vlm_sampler = VLMSampler(
    transformer=gemma3,
    tokenizer=gemma_tokenizer,
    image_size=image_size,
)
print("VLMSampler ready. pad_id:", vlm_sampler.pad_id(), "eos_id:", vlm_sampler.eos_id())

# --- (Optional) 10-second smoke test with a dummy image ---
import numpy as np, jax.numpy as jnp
dummy = np.zeros((1, image_size, image_size, 3), dtype=np.uint8)
out = vlm_sampler(
    input_strings=["Describe the image:"],
    images=jnp.asarray(dummy),
    max_generation_steps=8,
    temperature=0.0,
    return_logits=False,
    echo=False,
)
print(out.text[0])

In [ ]:
def get_lora_model(base_model, mesh):
  lora_provider = qwix.LoraProvider(
      module_path=(
          ".*q_einsum|.*kv_einsum|.*gate_proj|.*down_proj|.*up_proj|"
          ".*attn_vec_einsum"
      ),
      rank=RANK,
      alpha=ALPHA,
      #weight_qtype="nf4",
      #tile_size=4,
  )

  model_input = base_model.get_model_input()
  lora_model = qwix.apply_lora_to_model(
      base_model, lora_provider, **model_input
  )

  with mesh:
    state = nnx.state(lora_model)
    pspecs = nnx.get_partition_spec(state)
    sharded_state = jax.lax.with_sharding_constraint(state, pspecs)
    nnx.update(lora_model, sharded_state)

  return lora_model

In [ ]:
# Policy model
lora_gemma = get_lora_model(gemma3, mesh=mesh)
nnx.display(lora_gemma)

In [ ]:
from datasets import load_dataset
import numpy as np, jax.numpy as jnp
from PIL import Image
from tunix.models.siglip import preprocess as siglip_pp

# ==== 3) Load only the tokenizer first (cheap), NOT the whole processor yet ====
# Assuming gemma_tokenizer is already defined and is an instance of HFTokenizerAdapter
# If not, you would need to load it here or in a previous cell.
# from transformers import AutoTokenizer
# tok = AutoTokenizer.from_pretrained("google/paligemma-3b-mix-224", token=HF_TOKEN)
# class HFTokenizerAdapter: ... # Define the adapter if not already defined
# gemma_tokenizer = HFTokenizerAdapter(tok)


SPLIT = "train[:5000]"
MAX_PROMPT_LEN = 128
IMAGE_SIZE = 224

ds = load_dataset("openbmb/RLAIF-V-Dataset", split=SPLIT, token=HF_TOKEN)
cols = ["image", "question", "chosen", "rejected"]
ds = ds.remove_columns([c for c in ds.column_names if c not in cols])

def _pick_one_image(img_field):
    """Return a single PIL.Image from the dataset's image field."""
    x = img_field
    if isinstance(x, list):
        if not x:  # empty list, skip later
            return None
        x = x[0]
    if isinstance(x, Image.Image):
        return x.convert("RGB")
    # HF 'Image' feature sometimes gives np.ndarray
    arr = np.array(x)
    if arr.ndim == 3:
        return Image.fromarray(arr).convert("RGB")
    return None

def preprocess_item(ex):
    img = _pick_one_image(ex["image"])
    if img is None:
        return {
            "pixel_values": None,
            "question": ex["question"],
            "chosen": ex["chosen"],
            "rejected": ex["rejected"],
        }
    arr = np.array(img, dtype=np.uint8)[None, ...]              # [1,H,W,3]
    px  = siglip_pp.preprocess(jnp.asarray(arr), IMAGE_SIZE)     # [1,S,S,3] float32
    return {
        "pixel_values": np.asarray(px[0]),                       # [S,S,3]
        "question": ex["question"],
        "chosen": ex["chosen"],
        "rejected": ex["rejected"],
    }

ds = ds.with_transform(preprocess_item)

PROMPT_LEN = 128
ANS_LEN    = 128
PAD = gemma_tokenizer.pad_id()
EOS = gemma_tokenizer.eos_id()

def _left_pad_np(ids, L, pad=PAD):
    # ids: list[int]
    ids = ids[-L:] if len(ids) > L else [pad] * (L - len(ids)) + ids
    return np.asarray(ids, dtype=np.int32)

def _right_pad_np(ids, L, pad=PAD):
    # ids: list[int]
    ids = ids[:L]
    ids = ids + [pad] * (L - len(ids))
    return np.asarray(ids, dtype=np.int32)

def numpy_batches_fixed(dataset, batch_size=1, shuffle=True, seed=0):
    idx = np.arange(len(dataset))
    if shuffle:
        rng = np.random.default_rng(seed); rng.shuffle(idx)
    buf = []
    for i in idx:
        ex = dataset[int(i)]
        if ex["pixel_values"] is None:
            continue
        buf.append(ex)
        if len(buf) == batch_size:
            qs  = [b["question"] for b in buf]
            chs = [b["chosen"]   for b in buf]
            rjs = [b["rejected"] for b in buf]

            q_tok  = [gemma_tokenizer.encode(x) for x in qs]
            ch_tok = [gemma_tokenizer.encode(x) + [EOS] for x in chs]
            rj_tok = [gemma_tokenizer.encode(x) + [EOS] for x in rjs]

            Q  = np.stack([_left_pad_np (ids, PROMPT_LEN) for ids in q_tok ], axis=0)  # [B,P]
            CH = np.stack([_right_pad_np(ids, ANS_LEN   ) for ids in ch_tok], axis=0)  # [B,A]
            RJ = np.stack([_right_pad_np(ids, ANS_LEN   ) for ids in rj_tok], axis=0)  # [B,A]
            PX = np.stack([b["pixel_values"] for b in buf], axis=0).astype(np.float32) # [B,S,S,3]

            # only now move to device
            yield {
                "prompt_ids":   jnp.asarray(Q),
                "chosen_ids":   jnp.asarray(CH),
                "rejected_ids": jnp.asarray(RJ),
                "pixel_values": jnp.asarray(PX),
            }
            buf = []
    if buf:
        qs  = [b["question"] for b in buf]
        chs = [b["chosen"]   for b in buf]
        rjs = [b["rejected"] for b in buf]
        q_tok  = [gemma_tokenizer.encode(x) for x in qs]
        ch_tok = [gemma_tokenizer.encode(x) + [EOS] for x in chs]
        rj_tok = [gemma_tokenizer.encode(x) + [EOS] for x in rjs]
        Q  = np.stack([_left_pad_np (ids, PROMPT_LEN) for ids in q_tok ], axis=0)
        CH = np.stack([_right_pad_np(ids, ANS_LEN   ) for ids in ch_tok], axis=0)
        RJ = np.stack([_right_pad_np(ids, ANS_LEN   ) for ids in rj_tok], axis=0)
        PX = np.stack([b["pixel_values"] for b in buf], axis=0).astype(np.float32)
        yield {
            "prompt_ids":   jnp.asarray(Q),
            "chosen_ids":   jnp.asarray(CH),
            "rejected_ids": jnp.asarray(RJ),
            "pixel_values": jnp.asarray(PX),
        }

# Smoke one batch
b0 = next(numpy_batches_fixed(ds, batch_size=4))
print("Batch pixels:", b0["pixel_values"].shape, "| B:", b0["prompt_ids"].shape[0])
print("Batch prompt_ids:",   b0["prompt_ids"].shape)
print("Batch chosen_ids:",   b0["chosen_ids"].shape)
print("Batch rejected_ids:", b0["rejected_ids"].shape)

In [ ]:
import jax, jax.numpy as jnp

def _make_pos_and_causal_mask(tokens: jnp.ndarray, pad_id: int):
    """
    tokens: [B, L] int32
    returns:
      positions: [B, L] int32
      attn_mask: [B, L, L] bool  (causal & padding-aware)
    """
    # padding mask
    valid = (tokens != pad_id)               # [B, L]
    # positions: 0.. for valid tokens, 0 for pads (as you had)
    positions = (jnp.cumsum(valid.astype(jnp.int32), axis=1) - 1) * valid.astype(jnp.int32)

    # causal lower-triangular (shared across batch)
    L = tokens.shape[1]
    causal = jnp.tril(jnp.ones((L, L), dtype=bool))  # [L, L]

    # combine: query must be valid, key must be valid, and causal
    # broadcasting to [B, L, L]
    attn_mask = (valid[..., None] & valid[:, None, :] & causal[None, ...])
    return positions.astype(jnp.int32), attn_mask

def _vlm_forward_and_cache(policy_mod, *, tokens: jnp.ndarray, pixel_values: jnp.ndarray, pad_id: int):
    tokens = tokens.astype(jnp.int32)
    pixel_values = pixel_values.astype(jnp.float32)

    positions, attn_mask = _make_pos_and_causal_mask(tokens, pad_id)  # <-- causal [B,L,L]
    cache_in = {}  # own the cache per call to avoid side effects

    logits, cache_out = policy_mod(
        last_tokens=tokens,
        positions=positions,
        cache=cache_in,
        attention_mask=attn_mask,     # <-- correct rank-3, causal
        pixel_values=pixel_values,     # ignored for text-only configs
        output_hidden_states=False,
    )
    return logits, cache_out

def _seq_logprob_batch(policy_mod, px, prompt_ids, answer_ids, pad_id):
    """Sum of token logprobs for answer (teacher forcing). Returns [B]."""
    x_ids = jnp.concatenate([prompt_ids, answer_ids[:, :-1]], axis=1)
    logits, _ = _vlm_forward_and_cache(policy_mod, tokens=x_ids, pixel_values=px, pad_id=pad_id)
    La = answer_ids.shape[1]
    ans_logits = logits[:, -La:, :]
    logp = jax.nn.log_softmax(ans_logits, axis=-1)
    tok_logp = jnp.take_along_axis(logp, answer_ids[..., None], axis=-1)[..., 0]
    ans_mask = (answer_ids != pad_id).astype(tok_logp.dtype)
    return (tok_logp * ans_mask).sum(axis=1)  # [B]

def dpo_loss_batch_and_aux(policy, ref, batch, pad_id: int, beta: float = 0.1):
    px = jnp.asarray(batch["pixel_values"], dtype=jnp.float32)
    q  = jnp.asarray(batch.get("prompt_ids",   batch.get("question")),   dtype=jnp.int32)
    ch = jnp.asarray(batch.get("chosen_ids",   batch.get("chosen")),     dtype=jnp.int32)
    rj = jnp.asarray(batch.get("rejected_ids", batch.get("rejected")),   dtype=jnp.int32)

    lp_ch = _seq_logprob_batch(policy, px, q, ch, pad_id)   # [B]
    lp_rj = _seq_logprob_batch(policy, px, q, rj, pad_id)
    lq_ch = _seq_logprob_batch(ref,    px, q, ch, pad_id)
    lq_rj = _seq_logprob_batch(ref,    px, q, rj, pad_id)

    pol_margin = lp_ch - lp_rj                    # [B]
    ref_margin = lq_ch - lq_rj                    # [B]
    advantages = pol_margin - ref_margin          # [B]
    loss = -jax.nn.log_sigmoid(beta * advantages).mean()

    # per-batch metrics
    correct_mask = (pol_margin > 0)               # [B]
    num_correct = correct_mask.sum()
    B = q.shape[0]

    aux = {
        "loss": loss,
        "batch_acc": (num_correct / B),
        "num_correct": num_correct,
        "batch_size": B,
        "policy_margin_mean": pol_margin.mean(),
        "ref_margin_mean": ref_margin.mean(),
        "advantage_mean": advantages.mean(),
    }
    return loss, aux

In [ ]:
from flax import nnx
import optax


# immutable templates (never mutated)
policy_template = nnx.clone(lora_gemma)   # trainable LoRA-wrapped model
ref_template    = nnx.clone(gemma3)       # frozen reference

# live states
policy_state = nnx.state(lora_gemma)
ref_state    = nnx.state(gemma3)

PAD_ID = gemma_tokenizer.pad_id()
LR     = 2e-6
BETA   = 0.1
MAX_GRAD_NORM = 0.1

tx = optax.chain(
    optax.clip_by_global_norm(MAX_GRAD_NORM),
    optax.adamw(LR, b1=0.9, b2=0.99, weight_decay=0.1),
)
opt_state = tx.init(policy_state)

def loss_with_state(p_state, batch_arrays):
    pol  = nnx.clone(policy_template); nnx.update(pol, p_state)
    refm = nnx.clone(ref_template);    nnx.update(refm, ref_state)
    loss, aux = dpo_loss_batch_and_aux(pol, refm, batch_arrays, pad_id=PAD_ID, beta=BETA)
    new_p_state = nnx.state(pol)
    return loss, (new_p_state, aux)

@jax.jit
def train_step(policy_state, opt_state, batch_arrays):
    (loss, (policy_state_out, aux)), grads = jax.value_and_grad(
        loss_with_state, has_aux=True
    )(policy_state, batch_arrays)

    updates, opt_state2 = tx.update(grads, opt_state, params=policy_state_out)
    policy_state2 = optax.apply_updates(policy_state_out, updates)
    return loss, policy_state2, opt_state2, aux


In [ ]:
import time, matplotlib.pyplot as plt

steps = 200
log_every = 5


# per-step histories
loss_hist = []
batch_acc_hist = []
polm_hist = []
adv_hist  = []

# cumulative trackers
cum_correct = 0.0
cum_seen    = 0.0
cum_polm_sum = 0.0
cum_adv_sum  = 0.0
cum_acc_hist = []      # for plotting cumulative accuracy
cum_polm_hist = []     # cumulative mean policy margin
cum_adv_hist  = []     # cumulative mean advantage

itr = numpy_batches_fixed(ds, batch_size=16, shuffle=True, seed=42)
t0 = time.time()

for step in range(1, steps + 1):
    batch = next(itr)
    loss, policy_state, opt_state, aux = train_step(policy_state, opt_state, batch)

    # per-step logs
    loss_hist.append(float(loss))
    batch_acc_hist.append(float(aux["batch_acc"]))
    polm_hist.append(float(aux["policy_margin_mean"]))
    adv_hist.append(float(aux["advantage_mean"]))

    # update cumulative stats
    cum_correct += float(aux["num_correct"])
    cum_seen    += float(aux["batch_size"])
    cum_polm_sum += float(aux["policy_margin_mean"]) * float(aux["batch_size"])
    cum_adv_sum  += float(aux["advantage_mean"])     * float(aux["batch_size"])

    cum_acc = cum_correct / max(1.0, cum_seen)
    cum_polm = cum_polm_sum / max(1.0, cum_seen)
    cum_adv  = cum_adv_sum  / max(1.0, cum_seen)

    cum_acc_hist.append(cum_acc)
    cum_polm_hist.append(cum_polm)
    cum_adv_hist.append(cum_adv)

    if step % log_every == 0 or step == 1:
        dt = time.time() - t0
        print(f"step {step:4d}/{steps}  "
              f"loss={loss_hist[-1]:.4f}  "
              f"batch_acc={batch_acc_hist[-1]:.3f}  "
              f"cum_acc={cum_acc:.3f}  "
              f"polΔ={polm_hist[-1]:+.3f}  "
              f"adv={adv_hist[-1]:+.3f}  "
              f"({step/dt:.2f} steps/s)")

# final sync
nnx.update(lora_gemma, policy_state)

# --- plots ---
plt.figure(); plt.plot(loss_hist); plt.title("Train loss"); plt.xlabel("step"); plt.ylabel("loss"); plt.show()

plt.figure(); plt.plot(batch_acc_hist, label="batch acc"); plt.plot(cum_acc_hist, label="cumulative acc")
plt.title("Preference accuracy"); plt.xlabel("step"); plt.ylabel("accuracy"); plt.ylim(0,1); plt.legend(); plt.show()

plt.figure(); plt.plot(polm_hist, label="batch policy margin"); plt.plot(cum_polm_hist, label="cumulative policy margin")
plt.plot(adv_hist, label="batch advantage"); plt.plot(cum_adv_hist, label="cumulative advantage")
plt.title("Margins"); plt.xlabel("step"); plt.ylabel("nats"); plt.legend(); plt.show()